In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import networkx as nx
import scipy.sparse as sp
from random import randint
from sklearn.decomposition import PCA
#another PCA download
import time
from gensim.models import Word2Vec
from sklearn.metrics import mean_squared_error
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
import nltk
import json

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold
from scipy.sparse import hstack

import re
import lightgbm as lgb
from nltk.corpus import stopwords

# Deepwalk on matrix structure

In [4]:
# exact code from the last seminar
def random_walk(G, node, walk_length):
    # Simulates a random walk of length "walk_length" starting from node "node"

    #Please insert your code for Task 1 here
    walk = [node]
    for i in range(walk_length):
        neighbors =  list(G.neighbors(walk[i]))
        walk.append(neighbors[randint(0,len(neighbors)-1)])

    walk = [str(node) for node in walk]
    return walk

def generate_walks(G, num_walks, walk_length):
    # Runs "num_walks" random walks from each node

    walks = []

    #Please insert your code for Task 1 here
    for i in range(num_walks):
        permuted_nodes = np.random.permutation(G.nodes())
        for node in permuted_nodes:
            walks.append(random_walk(G, node, walk_length))

    return walks

def deepwalk(G, num_walks, walk_length, n_dim):
    # Simulates walks and uses the Skipgram model to learn node representations

    print("Generating walks")
    walks = generate_walks(G, num_walks, walk_length)

    print("Training word2vec")
    
    #Please insert your code for Task 2 here
    model = Word2Vec(vector_size=n_dim, window=8, min_count=0, sg=1, workers=8, hs=1)
    
    
    model.build_vocab(walks)
    model.train(walks, total_examples=model.corpus_count, epochs=5)

    return model

In [3]:
# read training data
df_train = pd.read_csv('train.csv', dtype={'author': np.int64, 'hindex': np.float32})
n_train = df_train.shape[0]

# read test data
df_test = pd.read_csv('test.csv', dtype={'author': np.int64})
n_test = df_test.shape[0]

# load the graph    
G = nx.read_edgelist('coauthorship.edgelist', delimiter=' ', nodetype=int)
n_nodes = G.number_of_nodes()
n_edges = G.number_of_edges() 
print('Number of nodes:', n_nodes)
print('Number of edges:', n_edges)
core_number = nx.core_number(G)
pr = nx.pagerank(G)
for el in pr:
  pr[el] = pr[el]*1000000

Number of nodes: 217801
Number of edges: 1718164


In [ ]:
start = time.time()
n_dim = 50
n_walks = 5
walk_length = 8
model = deepwalk(G, n_walks, walk_length, n_dim) 
print(time.time()-start, "time for deep walk")


In [ ]:
# Calculating on n_dim = 50, n_walks = 5, walk_length = 8  413 seconds
# Calculating on n_dim = 64, n_walks = 5, walk_length = 14  585 seconds
# Calculating on n_dim = 80, n_walks = 7, walk_length = 20  585 seconds

In [33]:
DeepWalk_embeddings = np.empty(shape=(df_train.shape[0]+df_test.shape[0], n_dim))
for idx,node in enumerate(G.nodes()):
    DeepWalk_embeddings[idx,:] = model.wv[str(node)]

nodes_graph = list(G.nodes())
vec_embed = {} # nodes corresponds to deepwalks_embeddings
for el in range(len(nodes_graph)):
    vec_embed[nodes_graph[el]] = DeepWalk_embeddings[el] 

In [4]:
X_train = np.zeros((n_train, 5))
y_train = np.zeros(n_train)
for i,row in df_train.iterrows():
    node = row['author']
    X_train[i, 0] = node
    X_train[i,1] = G.degree(node)
    X_train[i,2] = core_number[node]
    X_train[i,3] = pr[node]
    y_train[i] = row['hindex']

# create the test matrix. each node is represented as a vector of 3 features:
# (1) its degree, (2) its core number
X_test = np.zeros((n_test, 5))
for i,row in df_test.iterrows():
    node = row['author']
    X_test[i, 0] = node
    X_test[i,1] = G.degree(node)
    X_test[i,2] = core_number[node]
    X_test[i,3] = pr[node]
    
X_train_test = np.concatenate((X_train, X_test), axis = 0)

In [34]:
arr_vec_embed = []
for i,row in df_train.iterrows():
    node = row['author']
    X_train[i, 0] = node
    arr_vec_embed.append(vec_embed[node])
for i,row in df_test.iterrows():
    node = row['author']
    X_test[i, 0] = node
    arr_vec_embed.append(vec_embed[node])
    
    
df_arr= pd.DataFrame(data=arr_vec_embed) #dataframe of deepwalk embeddings

vec_embed = {}
arr_vec_embed = []
pr = []
model = 0
DeepWalk_embeddings = 0

In [35]:
#df_arr.to_pickle('deep_walk50')

In [239]:
from scipy.stats.stats import pearsonr   
  
pearsonr(X_train[:, 1],y_train) # correlation of degree

(0.4063264559353831, 0.0)

In [240]:
pearsonr(X_train[:, 2],y_train) # corrrelation of core_number

(0.3590667720340064, 0.0)

In [241]:
pearsonr(X_train[:, 3],y_train) #pagerank correlation

(0.40216429459635095, 0.0)

In [245]:
pearsonr(np.squeeze(mass_1[0:y_train.shape[0]]),y_train) # for average h_index

(0.3631428505021522, 0.0)

# Reading data from files

In [5]:
import time
start = time.time()
with open('abstracts.txt') as f:
    lines1 = f.readlines()
arr2 = []
indeces = []
for elem in range(0, len(lines1)):
    if elem % 100000 == 0:
        print(elem)
    mass = lines1[elem]
    indeces.append(int(mass.split('----')[0]))
    j = mass.split('----')[1]
    try:
        d = json.loads(j)
        mass1 = d['InvertedIndex']
        dl = d['IndexLength']
        arr1 = np.array(['ookdfnjdfmkdf' for _ in range(dl)])
        for el in mass1:
            for el1 in mass1[el]:
                arr1[el1] = el
    #arr2.append(arr1)
        arr2.append(' '.join(arr1))
    except ValueError:
        arr2.append(' ')
print(time.time()-start)

start = time.time()
abstracts = pd.DataFrame(index = indeces, data =arr2)
abstracts = abstracts.reset_index(level = [0])
abstracts = abstracts.set_axis(['paper_id', 'article'], axis='columns', inplace=False)
#abstracts.to_pickle('abstracts_table') # to save file
#abstracts = pd.read_pickle('abstracts_table') # to read file

0
100000
200000
300000
400000
500000
600000
119.17308807373047


In [6]:
d = {}
for el in range(len(indeces)):
    d[indeces[el]] = arr2[el]

In [7]:
with open('author_papers.txt') as f:
    lines1 = f.readlines()
    
indic = []
ids = []
d_num_papers = {}
arr = []
for i in range(0,len(lines1) ):
    cur_line = lines1[i].split(':')
    indic.append(int(cur_line[0]))
    d_num_papers[int(cur_line[0])] = len(cur_line[1].split('\n')[0].split('-'))
    arr.append(len(cur_line[1].split('\n')[0].split('-')))
    ids.append(cur_line[1].split('\n')[0].split('-'))

In [507]:
d1 = {'id':[1036332], 'article1': [1510273386], 'article2': [1827736641], 'article3':[1588673897], 'article4':[2252711322], 'article5':[2123653597]}
df_articles = pd.DataFrame(data=d1)

for i in range(0, len(ids)):
    cur_arr = np.zeros(6)
    cur_arr[0] = indic[i] 
    for j in range(len(ids[i])):
        #new_row = {'id':indic[i], 'articles':ids[i][j]} 
        cur_arr[j+1] = ids[i][j]
        
        
        #df_articles = df_articles.append(new_row, ignore_index=True)
        df_articles.loc[i] = cur_arr
    if i % 50000 == 0:
        print(i)

0


In [8]:
#df_articles = pd.read_pickle('articles_table')
df_articles.head()

,id,articles1,articles2,articles3,articles4,articles5
0,1036332.0,1.510273e+09,1.827737e+09,1.588674e+09,2.252711e+09,2.123654e+09
2,1101850.0,1.334590e+08,1.797197e+08,2.111788e+09,2.126489e+09,3.183900e+07
3,1336878.0,2.122092e+09,2.132110e+09,2.100272e+09,2.065673e+09,2.036414e+09
4,1515524.0,2.141828e+09,2.127086e+09,2.013548e+09,2.138530e+09,1.994864e+09
5,1606427.0,1.907725e+09,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00


In [16]:
#abstracts.to_pickle('articles_table') # to save file
#df_articles = pd.read_pickle('articles_table') # to read from file

In [9]:
id_paper_first = df_articles[['articles1']].to_numpy()
author_papers_id = df_articles[['id']].to_numpy()
id_paper_second = df_articles[['articles2']].to_numpy()
id_paper_third = df_articles[['articles3']].to_numpy()
id_paper_four = df_articles[['articles4']].to_numpy()
id_paper_five = df_articles[['articles5']].to_numpy()

In [10]:
d_author_id_article1= {}
d_author_id_article2= {}
d_author_id_article3= {}
d_author_id_article4= {}
d_author_id_article5= {}
for i in range(len(id_paper_first)):
    d_author_id_article1[author_papers_id[i][0]] = id_paper_first[i][0]
    d_author_id_article2[author_papers_id[i][0]] = id_paper_second[i][0]
    d_author_id_article3[author_papers_id[i][0]] = id_paper_third[i][0]
    d_author_id_article4[author_papers_id[i][0]] = id_paper_four[i][0]
    d_author_id_article5[author_papers_id[i][0]] = id_paper_five[i][0]

In [11]:
first_article = []
second_article = []
third_article = []
four_article = []
five_article = []
fl = 0
for i in range(X_train_test.shape[0]):
    try:
        el1 = d[d_author_id_article1[X_train_test[i, 0]]]
        el2 = d[d_author_id_article2[X_train_test[i, 0]]]
        el3 = d[d_author_id_article3[X_train_test[i, 0]]]
        el4 = d[d_author_id_article4[X_train_test[i, 0]]]
        el5 = d[d_author_id_article5[X_train_test[i, 0]]]
        first_article.append(el1)
        second_article.append(el2)
        third_article.append(el3)
        four_article.append(el4)
        five_article.append(el5)
        
    except KeyError:
        first_article.append(' ')
        second_article.append(' ')
        third_article.append(' ')
        four_article.append(' ')
        five_article.append(' ')
        fl += 1
    if i % 20000 == 0:
        print(i)
        
# erase data, because dict use much memory
d_author_id_article1= {}
d_author_id_article2= {}
d_author_id_article3= {}
d_author_id_article4= {}
d_author_id_article5= {}

0
20000
40000
60000
80000
100000
120000
140000
160000
180000
200000


In [12]:
d_train_main  = {'id':X_train[:, 0], 'article_1': first_article[0:174241], 'article_2': second_article[0:174241], 'article_3':third_article[0:174241], 
                 'article_4':four_article[0:174241], 'article_5':five_article[0:174241]}
df_train_main = pd.DataFrame(data=d_train_main)

In [13]:
d_test_main = {'id':X_test[:, 0], 'article_1': first_article[174241:], 'article_2': second_article[174241:], 'article_3':third_article[174241:], 
                 'article_4':four_article[174241:], 'article_5':five_article[174241:]}
df_test_main = pd.DataFrame(data=d_test_main)


#df_train_main.to_pickle('train_article') # save to the file
#df_test_main.to_pickle('test_article')

#df_train_main = pd.read_pickle('train_article') #read from the file
#df_test_main = pd.read_pickle('test_article')

# Making features using word2vec

In [ ]:
stop_words = stopwords.words('english')

def clean_sentence(sentence):
    sentence = sentence.lower().strip()
    sentence = re.sub(r"(@\[A-Za-z]+)|([^A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", sentence)
    #regular expression which deletes numbers 
    
    #return " ".join([word for word in sentence.split() if word not in (stop_words)])
    return sentence

In [17]:

df_train_main['articles_tog'] = '' #sum all articles together
cur_mass = []
for i in range(df_train_main.shape[0]):
    cur_mass.append(df_train_main.loc[i].article_1 + df_train_main.loc[i].article_2+ df_train_main.loc[i].article_3+df_train_main.loc[i].article_4+df_train_main.loc[i].article_5)
    if i % 50000 == 0:
        print(i, "Train")

df_train_main['articles_tog'] = cur_mass
#df_train_main.to_pickle('train_article') # save results to df_train_main


df_test_main['articles_tog'] = '' #same for test
cur_mass = []
for i in range(df_test_main.shape[0]):
    cur_mass.append(df_test_main.loc[i].article_1 + df_test_main.loc[i].article_2+ df_test_main.loc[i].article_3+df_test_main.loc[i].article_4+df_test_main.loc[i].article_5)
    if i % 50000 == 0:
        print(i, "Test")

df_test_main['articles_tog'] = cur_mass
#df_test_main.to_pickle('test_article') 

0 Train
0 Test


In [ ]:
# articles_tog -> articles with summed all articles
start = time.time()
testing_clean = []

for i in range(df_train_main.shape[0]):
    sentence = clean_sentence(df_train_main.iloc[i].articles_tog)
    sentence = " ".join([word for word in sentence.split() if word not in (stop_words)])
    testing_clean.append(sentence)
    
for i in range(df_test_main.shape[0]):
    sentence = clean_sentence(df_test_main.iloc[i].articles_tog)
    sentence = " ".join([word for word in sentence.split() if word not in (stop_words)])
    testing_clean.append(sentence)
    
print(time.time()-start, "time for cleaning")

wpt = nltk.WordPunctTokenizer()
tokenized_corpus = [wpt.tokenize(document) for document in testing_clean]

start = time.time()
# Set values for various parameters
feature_size = 100    # Word vector dimensionality  
window_context = 30   # Context window size                                                                                    
min_word_count = 1    # Minimum word count                        
sample = 1e-3   # Downsample setting for frequent words

w2v_model = word2vec.Word2Vec(tokenized_corpus, size=feature_size, 
                          window=window_context, min_count=min_word_count,
                          sample=sample, iter=50)
print(time.time()-start, "Time for word2vec model")

def average_word_vectors(words, model, vocabulary, num_features):
    
    feature_vector = np.zeros((num_features,),dtype="float64")
    nwords = 0.
    
    for word in words:
        if word in vocabulary: 
            nwords = nwords + 1.
            feature_vector = np.add(feature_vector, model[word])
    
    if nwords:
        feature_vector = np.divide(feature_vector, nwords)
        
    return feature_vector
    
   
def averaged_word_vectorizer(corpus, model, num_features):
    vocabulary = set(model.wv.index2word)
    features = [average_word_vectors(tokenized_sentence, model, vocabulary, num_features)
                    for tokenized_sentence in corpus]
    return np.array(features)


# get document level embeddings
start = time.time()
w2v_feature_array = averaged_word_vectorizer(corpus=tokenized_corpus, model=w2v_model,
                                             num_features=feature_size)
w2v_feature_array_1 = pd.DataFrame(w2v_feature_array) # Save data for word2vec features 
w2v_feature_array_1.to_pickle('w2v_feature_array_1')

#features_word2vec = pd.read_pickle('w2v_feature_array_1') #upload file 
print(time.time()-start)

# Creating new features

* 1 if author has 5 pages
* average h_index of neigbours for each author
* max_h_index of neighbours
* min h_index of neighbours

In [15]:
with open('author_papers.txt') as f:
    lines1 = f.readlines()
    
d_num_papers = {}
for i in range(0,len(lines1) ):
    cur_line = lines1[i].split(':')
    d_num_papers[int(cur_line[0])] = len(cur_line[1].split('\n')[0].split('-'))


In [16]:
new_features_5 = [] # feature 1 if author has 5 pages and 0 otherwise
for i in range(X_train_test.shape[0]):
    el = d_num_papers[X_train_test[i][0]]
    if el == 5:
        new_features_5.append(1)
    else:
        new_features_5.append(0)

In [17]:
d = {} # store for author -> h_index
for i in range(X_train.shape[0]):
    d[X_train[i][0]] = y_train[i]

mass_1 = [] # feature for calculating average h_index
start = time.time()
for i in range(X_train_test.shape[0]):
    
    neighb = list(G.neighbors(X_train_test[i][0]))
    ch = 0
    col = 0
    for neigh in neighb:
    
        if (neigh in d) == False:
            ch += 0
        else:
            ch += float(d[neigh])
            col += 1
    if col > 0:
        mass_1.append(ch/col)
    else:
        mass_1.append(0)
print(time.time()-start, "average h_index for neighbours")

mass_1 = np.array(mass_1)
mass_1 = mass_1.reshape(-1, 1)
new_features_5 = np.array(new_features_5)
new_features_5 = new_features_5.reshape(-1, 1)

3.3082919120788574 average h_index for neighbours


In [21]:
mass_maxi = []
mass_mini = []
start = time.time()
for i in range(X_train_test.shape[0]): 
    
    neighb = list(G.neighbors(X_train_test[i][0]))
    maxi = 0
    mini = 400
    for neigh in neighb:
        
        
        if (neigh in d) == False:
            maxi += 0
            mini += 0
        else:
            if  float(d[neigh]) > maxi:
                maxi = float(d[neigh])
        
            if  float(d[neigh]) < mini:
                mini = float(d[neigh])
                
            
    if mini == 400:
        mass_mini.append(0)
    else:
        mass_mini.append(mini)
    mass_maxi.append(maxi)

print(time.time()-start)

mass_maxi = np.array(mass_maxi)
mass_maxi = mass_maxi.reshape(-1, 1)
mass_mini = np.array(mass_mini)
mass_mini = mass_mini.reshape(-1, 1)

4.661071062088013


In [18]:
features_word2vec = pd.read_pickle('w2v_feature_array_1') # There are the features of word2vec in that file
'''abstracts = pd.read_pickle('abstracts_table')
df_articles = pd.read_pickle('articles_table')
tfidf_vector = pd.read_pickle('author_tfidf_vector.txt')
df_arr = pd.read_pickle('deep_walk')'''

"abstracts = pd.read_pickle('abstracts_table')\ndf_articles = pd.read_pickle('articles_table')\ntfidf_vector = pd.read_pickle('author_tfidf_vector.txt')\ndf_arr = pd.read_pickle('deep_walk')"

In [508]:
final_test = np.hstack((X_train[:, 1:4], mass_1[0:174241],  new_features_5[0:174241], features_word2vec[0:174241], df_arr[0:174241]))
final_test_test = np.hstack((X_test[:, 1:4],mass_1[174241:], new_features_5[174241:], features_word2vec[174241:],  df_arr[174241:]))

In [43]:
from catboost import CatBoostRegressor

In [420]:
train1, test1, y_train1, y_test1 = train_test_split(final_test, y_train, test_size=.33, random_state = 42)
start = time.time()
clf = CatBoostRegressor(n_estimators = 1000,learning_rate = 0.2,  depth = 5)
clf.fit(train1, y_train1)
y_pred = clf.predict(test1)
print(time.time()-start)

0:	learn: 11.5906963	total: 45.8ms	remaining: 45.8s
1:	learn: 10.8705751	total: 78.8ms	remaining: 39.3s
2:	learn: 10.2954572	total: 106ms	remaining: 35.3s
3:	learn: 9.8548254	total: 134ms	remaining: 33.3s
4:	learn: 9.5276132	total: 164ms	remaining: 32.7s
5:	learn: 9.2339117	total: 197ms	remaining: 32.6s
6:	learn: 9.0262557	total: 226ms	remaining: 32.1s
7:	learn: 8.8509362	total: 270ms	remaining: 33.4s
8:	learn: 8.7148745	total: 320ms	remaining: 35.2s
9:	learn: 8.6088308	total: 349ms	remaining: 34.5s
10:	learn: 8.5224748	total: 381ms	remaining: 34.3s
11:	learn: 8.4500301	total: 408ms	remaining: 33.6s
12:	learn: 8.3851216	total: 437ms	remaining: 33.2s
13:	learn: 8.3246495	total: 479ms	remaining: 33.7s
14:	learn: 8.2732097	total: 524ms	remaining: 34.4s
15:	learn: 8.2275687	total: 566ms	remaining: 34.8s
16:	learn: 8.1780858	total: 605ms	remaining: 35s
17:	learn: 8.1296334	total: 634ms	remaining: 34.6s
18:	learn: 8.0991608	total: 661ms	remaining: 34.1s
19:	learn: 8.0720488	total: 703ms	rema

165:	learn: 6.9504704	total: 5.79s	remaining: 29.1s
166:	learn: 6.9462694	total: 5.82s	remaining: 29s
167:	learn: 6.9423889	total: 5.85s	remaining: 29s
168:	learn: 6.9393286	total: 5.89s	remaining: 29s
169:	learn: 6.9357348	total: 5.92s	remaining: 28.9s
170:	learn: 6.9325218	total: 5.95s	remaining: 28.9s
171:	learn: 6.9280211	total: 5.98s	remaining: 28.8s
172:	learn: 6.9244322	total: 6.02s	remaining: 28.8s
173:	learn: 6.9215545	total: 6.05s	remaining: 28.7s
174:	learn: 6.9185464	total: 6.08s	remaining: 28.7s
175:	learn: 6.9142221	total: 6.11s	remaining: 28.6s
176:	learn: 6.9079157	total: 6.14s	remaining: 28.5s
177:	learn: 6.9047058	total: 6.17s	remaining: 28.5s
178:	learn: 6.9008308	total: 6.2s	remaining: 28.4s
179:	learn: 6.8973420	total: 6.24s	remaining: 28.4s
180:	learn: 6.8939245	total: 6.27s	remaining: 28.4s
181:	learn: 6.8900173	total: 6.3s	remaining: 28.3s
182:	learn: 6.8852714	total: 6.33s	remaining: 28.3s
183:	learn: 6.8799073	total: 6.36s	remaining: 28.2s
184:	learn: 6.876517

329:	learn: 6.4655319	total: 11s	remaining: 22.3s
330:	learn: 6.4620754	total: 11s	remaining: 22.3s
331:	learn: 6.4598298	total: 11s	remaining: 22.2s
332:	learn: 6.4574603	total: 11.1s	remaining: 22.2s
333:	learn: 6.4550916	total: 11.1s	remaining: 22.1s
334:	learn: 6.4539872	total: 11.1s	remaining: 22.1s
335:	learn: 6.4526886	total: 11.2s	remaining: 22s
336:	learn: 6.4511237	total: 11.2s	remaining: 22s
337:	learn: 6.4488183	total: 11.2s	remaining: 22s
338:	learn: 6.4465880	total: 11.3s	remaining: 21.9s
339:	learn: 6.4444646	total: 11.3s	remaining: 21.9s
340:	learn: 6.4424049	total: 11.3s	remaining: 21.9s
341:	learn: 6.4401475	total: 11.3s	remaining: 21.8s
342:	learn: 6.4368860	total: 11.4s	remaining: 21.8s
343:	learn: 6.4347956	total: 11.4s	remaining: 21.7s
344:	learn: 6.4328041	total: 11.4s	remaining: 21.7s
345:	learn: 6.4309286	total: 11.5s	remaining: 21.7s
346:	learn: 6.4287616	total: 11.5s	remaining: 21.6s
347:	learn: 6.4271909	total: 11.5s	remaining: 21.6s
348:	learn: 6.4254868	to

493:	learn: 6.1347183	total: 16.4s	remaining: 16.8s
494:	learn: 6.1334030	total: 16.5s	remaining: 16.8s
495:	learn: 6.1317006	total: 16.5s	remaining: 16.8s
496:	learn: 6.1293980	total: 16.5s	remaining: 16.7s
497:	learn: 6.1272778	total: 16.6s	remaining: 16.8s
498:	learn: 6.1262147	total: 16.7s	remaining: 16.8s
499:	learn: 6.1242588	total: 16.7s	remaining: 16.7s
500:	learn: 6.1216946	total: 16.8s	remaining: 16.7s
501:	learn: 6.1200068	total: 16.8s	remaining: 16.7s
502:	learn: 6.1184979	total: 16.8s	remaining: 16.6s
503:	learn: 6.1169504	total: 16.9s	remaining: 16.6s
504:	learn: 6.1158221	total: 16.9s	remaining: 16.6s
505:	learn: 6.1143737	total: 17s	remaining: 16.6s
506:	learn: 6.1126798	total: 17s	remaining: 16.5s
507:	learn: 6.1106637	total: 17.1s	remaining: 16.5s
508:	learn: 6.1095189	total: 17.1s	remaining: 16.5s
509:	learn: 6.1079596	total: 17.1s	remaining: 16.4s
510:	learn: 6.1058159	total: 17.2s	remaining: 16.4s
511:	learn: 6.1036315	total: 17.2s	remaining: 16.4s
512:	learn: 6.10

655:	learn: 5.8695192	total: 26.5s	remaining: 13.9s
656:	learn: 5.8679557	total: 26.6s	remaining: 13.9s
657:	learn: 5.8658609	total: 26.6s	remaining: 13.8s
658:	learn: 5.8644360	total: 26.7s	remaining: 13.8s
659:	learn: 5.8631110	total: 26.7s	remaining: 13.7s
660:	learn: 5.8616805	total: 26.7s	remaining: 13.7s
661:	learn: 5.8596323	total: 26.8s	remaining: 13.7s
662:	learn: 5.8581047	total: 26.8s	remaining: 13.6s
663:	learn: 5.8569230	total: 26.9s	remaining: 13.6s
664:	learn: 5.8552487	total: 26.9s	remaining: 13.6s
665:	learn: 5.8544169	total: 26.9s	remaining: 13.5s
666:	learn: 5.8530652	total: 27s	remaining: 13.5s
667:	learn: 5.8515544	total: 27s	remaining: 13.4s
668:	learn: 5.8501769	total: 27.1s	remaining: 13.4s
669:	learn: 5.8481273	total: 27.1s	remaining: 13.4s
670:	learn: 5.8468226	total: 27.2s	remaining: 13.3s
671:	learn: 5.8452494	total: 27.2s	remaining: 13.3s
672:	learn: 5.8443814	total: 27.2s	remaining: 13.2s
673:	learn: 5.8428030	total: 27.3s	remaining: 13.2s
674:	learn: 5.84

815:	learn: 5.6536503	total: 34.7s	remaining: 7.82s
816:	learn: 5.6525920	total: 34.7s	remaining: 7.78s
817:	learn: 5.6510187	total: 34.8s	remaining: 7.73s
818:	learn: 5.6493926	total: 34.8s	remaining: 7.69s
819:	learn: 5.6486647	total: 34.8s	remaining: 7.65s
820:	learn: 5.6474760	total: 34.9s	remaining: 7.6s
821:	learn: 5.6455209	total: 34.9s	remaining: 7.56s
822:	learn: 5.6441589	total: 34.9s	remaining: 7.52s
823:	learn: 5.6426594	total: 35s	remaining: 7.47s
824:	learn: 5.6412089	total: 35s	remaining: 7.43s
825:	learn: 5.6401398	total: 35s	remaining: 7.38s
826:	learn: 5.6387824	total: 35.1s	remaining: 7.34s
827:	learn: 5.6372854	total: 35.2s	remaining: 7.32s
828:	learn: 5.6364637	total: 35.3s	remaining: 7.27s
829:	learn: 5.6349348	total: 35.3s	remaining: 7.23s
830:	learn: 5.6335066	total: 35.3s	remaining: 7.18s
831:	learn: 5.6319334	total: 35.3s	remaining: 7.14s
832:	learn: 5.6298175	total: 35.4s	remaining: 7.09s
833:	learn: 5.6287342	total: 35.4s	remaining: 7.05s
834:	learn: 5.62763

975:	learn: 5.4591896	total: 40.9s	remaining: 1s
976:	learn: 5.4581738	total: 41s	remaining: 964ms
977:	learn: 5.4572790	total: 41s	remaining: 922ms
978:	learn: 5.4564295	total: 41s	remaining: 880ms
979:	learn: 5.4557047	total: 41.1s	remaining: 838ms
980:	learn: 5.4545691	total: 41.1s	remaining: 796ms
981:	learn: 5.4538125	total: 41.2s	remaining: 754ms
982:	learn: 5.4525895	total: 41.2s	remaining: 713ms
983:	learn: 5.4516590	total: 41.3s	remaining: 672ms
984:	learn: 5.4505337	total: 41.4s	remaining: 630ms
985:	learn: 5.4493175	total: 41.4s	remaining: 588ms
986:	learn: 5.4484054	total: 41.4s	remaining: 546ms
987:	learn: 5.4470536	total: 41.5s	remaining: 504ms
988:	learn: 5.4460948	total: 41.5s	remaining: 462ms
989:	learn: 5.4447021	total: 41.5s	remaining: 420ms
990:	learn: 5.4438223	total: 41.6s	remaining: 378ms
991:	learn: 5.4426905	total: 41.6s	remaining: 336ms
992:	learn: 5.4410142	total: 41.7s	remaining: 294ms
993:	learn: 5.4398617	total: 41.7s	remaining: 252ms
994:	learn: 5.4386112

In [ ]:
'''# Analyze for predictions and their distribution compared to real model
 10 < x  11 to 15.5 37743 to 38812 
10 < x < 20 77 to 39     12711 to 8963
20 < x < 30 170 to 86.66  4127 to 3391 observations
30  < x < 50  277 to 210 mse 2137 to 2484
50 < x < 80 276 to 489 752 to 1049
> 80 30 to 160 500 vs 2800 '''


# Checking quality of the model

In [ ]:
n_estim = [100, 200, 300]
learning = [0.1,0.12]
num_leave = [20, 50, 100, 200]


cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1121218)
for el in n_estim:
    cv_scores = np.empty(5)
    for idx, (train_idx, test_idx) in enumerate(cv.split(final_test, y_train)):
        X_train, X_test = final_test[train_idx], final_test[test_idx]
        y_train1, y_test1 = y_train[train_idx], y_train[test_idx]

        model = lgb.LGBMRegressor(n_estimators = 500, learning_rate = 0.12, num_leaves = 200)
        model.fit(X_train, y_train1)
        preds = model.predict(X_test)
        cv_scores[idx] = mean_squared_error(y_test1, preds)
    print(cv_scores,np.mean(cv_scores),  el)

In [ ]:
#n_estimators
'''[53.53571033 54.50651916 53.60322133] 100
[51.77734503 52.61463367 51.70348592] 200
[50.42872934 50.96430724 50.22325336] 500'''

#lr
#[51.38879843 52.64495793 51.90799202 51.37096127 50.84715438] 51.63197280673071 0.12
#num_leaves

#[52.96982495 53.52256497 52.76881649 52.04011541 51.48920142] 52.558104647817274 20
#[51.3099745  51.78890643 50.82470602 50.65195368 50.0970291 ] 50.93451394471755 50
#[50.16356784 51.15144598 50.80201971 49.90393926 49.69891136] 50.343976829826204 100
#[50.06984358 51.40634118 50.22492043 50.51083878 49.07635478] 50.25765974824528 200

# Use there final model with best hyperparameters

In [146]:
start = time.time()
reg = lgb.LGBMRegressor(n_estimators = 700,learning_rate = 0.12, num_leaves = 150)
reg.fit(final_test, y_train)
y_pred = reg.predict(final_test_test)

clf = CatBoostRegressor(n_estimators = 1000,learning_rate = 0.2,  depth = 7)
clf.fit(final_test, y_train)
y_pred1 = clf.predict(final_test_test)


clf1 = CatBoostRegressor(n_estimators = 1000,learning_rate = 0.2,  depth = 7, random_state = 54)
clf1.fit(final_test, y_train)
y_pred2 = clf1.predict(final_test_test)
print(time.time()-start)


0:	learn: 11.4765425	total: 76.6ms	remaining: 1m 16s
1:	learn: 10.6426250	total: 141ms	remaining: 1m 10s
2:	learn: 10.0113332	total: 199ms	remaining: 1m 6s
3:	learn: 9.5420733	total: 262ms	remaining: 1m 5s
4:	learn: 9.2088593	total: 356ms	remaining: 1m 10s
5:	learn: 8.9370196	total: 416ms	remaining: 1m 8s
6:	learn: 8.7329705	total: 489ms	remaining: 1m 9s
7:	learn: 8.5596911	total: 551ms	remaining: 1m 8s
8:	learn: 8.4248498	total: 639ms	remaining: 1m 10s
9:	learn: 8.3121877	total: 710ms	remaining: 1m 10s
10:	learn: 8.2322134	total: 774ms	remaining: 1m 9s
11:	learn: 8.1519623	total: 836ms	remaining: 1m 8s
12:	learn: 8.0873693	total: 912ms	remaining: 1m 9s
13:	learn: 8.0299645	total: 981ms	remaining: 1m 9s
14:	learn: 7.9821436	total: 1.06s	remaining: 1m 9s
15:	learn: 7.9373085	total: 1.16s	remaining: 1m 11s
16:	learn: 7.9030171	total: 1.3s	remaining: 1m 15s
17:	learn: 7.8681228	total: 1.41s	remaining: 1m 16s
18:	learn: 7.8376158	total: 1.47s	remaining: 1m 15s
19:	learn: 7.8097147	total: 1

159:	learn: 6.6470907	total: 13s	remaining: 1m 8s
160:	learn: 6.6424333	total: 13s	remaining: 1m 7s
161:	learn: 6.6390980	total: 13.1s	remaining: 1m 7s
162:	learn: 6.6353715	total: 13.2s	remaining: 1m 7s
163:	learn: 6.6298545	total: 13.3s	remaining: 1m 7s
164:	learn: 6.6247428	total: 13.4s	remaining: 1m 7s
165:	learn: 6.6201142	total: 13.5s	remaining: 1m 7s
166:	learn: 6.6155000	total: 13.6s	remaining: 1m 7s
167:	learn: 6.6117509	total: 13.7s	remaining: 1m 7s
168:	learn: 6.6060016	total: 13.8s	remaining: 1m 7s
169:	learn: 6.5998908	total: 13.8s	remaining: 1m 7s
170:	learn: 6.5965463	total: 13.9s	remaining: 1m 7s
171:	learn: 6.5911087	total: 14s	remaining: 1m 7s
172:	learn: 6.5855486	total: 14.1s	remaining: 1m 7s
173:	learn: 6.5818892	total: 14.2s	remaining: 1m 7s
174:	learn: 6.5771045	total: 14.3s	remaining: 1m 7s
175:	learn: 6.5725283	total: 14.3s	remaining: 1m 7s
176:	learn: 6.5676709	total: 14.4s	remaining: 1m 7s
177:	learn: 6.5631523	total: 14.5s	remaining: 1m 7s
178:	learn: 6.5582

319:	learn: 6.0561263	total: 26.1s	remaining: 55.4s
320:	learn: 6.0527203	total: 26.1s	remaining: 55.3s
321:	learn: 6.0505349	total: 26.2s	remaining: 55.2s
322:	learn: 6.0481734	total: 26.3s	remaining: 55s
323:	learn: 6.0459124	total: 26.4s	remaining: 55s
324:	learn: 6.0424304	total: 26.4s	remaining: 54.9s
325:	learn: 6.0389331	total: 26.5s	remaining: 54.8s
326:	learn: 6.0359600	total: 26.6s	remaining: 54.7s
327:	learn: 6.0334998	total: 26.7s	remaining: 54.6s
328:	learn: 6.0303512	total: 26.8s	remaining: 54.6s
329:	learn: 6.0275615	total: 26.8s	remaining: 54.4s
330:	learn: 6.0262057	total: 26.9s	remaining: 54.3s
331:	learn: 6.0230442	total: 26.9s	remaining: 54.2s
332:	learn: 6.0207791	total: 27s	remaining: 54.1s
333:	learn: 6.0183361	total: 27.1s	remaining: 54s
334:	learn: 6.0145105	total: 27.2s	remaining: 53.9s
335:	learn: 6.0120234	total: 27.3s	remaining: 53.9s
336:	learn: 6.0085245	total: 27.3s	remaining: 53.8s
337:	learn: 6.0048578	total: 27.4s	remaining: 53.7s
338:	learn: 6.002979

478:	learn: 5.6478044	total: 39.8s	remaining: 43.3s
479:	learn: 5.6454529	total: 39.9s	remaining: 43.2s
480:	learn: 5.6424665	total: 39.9s	remaining: 43.1s
481:	learn: 5.6399612	total: 40s	remaining: 43s
482:	learn: 5.6376892	total: 40.1s	remaining: 42.9s
483:	learn: 5.6347834	total: 40.2s	remaining: 42.8s
484:	learn: 5.6328717	total: 40.3s	remaining: 42.8s
485:	learn: 5.6310305	total: 40.3s	remaining: 42.7s
486:	learn: 5.6284574	total: 40.4s	remaining: 42.6s
487:	learn: 5.6267769	total: 40.5s	remaining: 42.4s
488:	learn: 5.6255057	total: 40.5s	remaining: 42.3s
489:	learn: 5.6225527	total: 40.6s	remaining: 42.2s
490:	learn: 5.6196958	total: 40.6s	remaining: 42.1s
491:	learn: 5.6180073	total: 40.7s	remaining: 42s
492:	learn: 5.6157451	total: 40.8s	remaining: 41.9s
493:	learn: 5.6132397	total: 40.9s	remaining: 41.8s
494:	learn: 5.6110123	total: 40.9s	remaining: 41.7s
495:	learn: 5.6085937	total: 41s	remaining: 41.7s
496:	learn: 5.6067877	total: 41.1s	remaining: 41.6s
497:	learn: 5.604042

638:	learn: 5.3234298	total: 54.1s	remaining: 30.6s
639:	learn: 5.3224636	total: 54.2s	remaining: 30.5s
640:	learn: 5.3208871	total: 54.2s	remaining: 30.4s
641:	learn: 5.3187772	total: 54.3s	remaining: 30.3s
642:	learn: 5.3165370	total: 54.4s	remaining: 30.2s
643:	learn: 5.3140864	total: 54.5s	remaining: 30.1s
644:	learn: 5.3116386	total: 54.6s	remaining: 30s
645:	learn: 5.3094159	total: 54.6s	remaining: 29.9s
646:	learn: 5.3071691	total: 54.7s	remaining: 29.9s
647:	learn: 5.3046267	total: 54.8s	remaining: 29.8s
648:	learn: 5.3028092	total: 54.9s	remaining: 29.7s
649:	learn: 5.3007056	total: 54.9s	remaining: 29.6s
650:	learn: 5.2987062	total: 55s	remaining: 29.5s
651:	learn: 5.2959803	total: 55.1s	remaining: 29.4s
652:	learn: 5.2947729	total: 55.2s	remaining: 29.3s
653:	learn: 5.2929158	total: 55.3s	remaining: 29.2s
654:	learn: 5.2912557	total: 55.3s	remaining: 29.1s
655:	learn: 5.2890114	total: 55.4s	remaining: 29s
656:	learn: 5.2869037	total: 55.5s	remaining: 29s
657:	learn: 5.286300

...

270:	learn: 6.2149046	total: 24.7s	remaining: 1m 6s
271:	learn: 6.2131812	total: 24.8s	remaining: 1m 6s
272:	learn: 6.2102972	total: 24.9s	remaining: 1m 6s
273:	learn: 6.2069450	total: 24.9s	remaining: 1m 6s
274:	learn: 6.2051544	total: 25s	remaining: 1m 5s
275:	learn: 6.2019765	total: 25.1s	remaining: 1m 5s
276:	learn: 6.1987764	total: 25.1s	remaining: 1m 5s
277:	learn: 6.1953667	total: 25.2s	remaining: 1m 5s
278:	learn: 6.1932690	total: 25.3s	remaining: 1m 5s
279:	learn: 6.1897403	total: 25.3s	remaining: 1m 5s
280:	learn: 6.1872193	total: 25.4s	remaining: 1m 5s
281:	learn: 6.1842436	total: 25.5s	remaining: 1m 4s
282:	learn: 6.1802926	total: 25.6s	remaining: 1m 4s
283:	learn: 6.1763822	total: 25.6s	remaining: 1m 4s
284:	learn: 6.1734085	total: 25.7s	remaining: 1m 4s
285:	learn: 6.1702376	total: 25.8s	remaining: 1m 4s
286:	learn: 6.1675922	total: 25.8s	remaining: 1m 4s
287:	learn: 6.1634022	total: 25.9s	remaining: 1m 4s
288:	learn: 6.1607796	total: 26s	remaining: 1m 3s
289:	learn: 6.15

430:	learn: 5.7775862	total: 35.8s	remaining: 47.2s
431:	learn: 5.7758721	total: 35.8s	remaining: 47.1s
432:	learn: 5.7741088	total: 35.9s	remaining: 47s
433:	learn: 5.7719773	total: 36s	remaining: 46.9s
434:	learn: 5.7705716	total: 36s	remaining: 46.8s
435:	learn: 5.7680849	total: 36.1s	remaining: 46.7s
436:	learn: 5.7658361	total: 36.2s	remaining: 46.6s
437:	learn: 5.7641049	total: 36.2s	remaining: 46.5s
438:	learn: 5.7619094	total: 36.3s	remaining: 46.4s
439:	learn: 5.7596922	total: 36.4s	remaining: 46.3s
440:	learn: 5.7575314	total: 36.4s	remaining: 46.2s
441:	learn: 5.7542563	total: 36.5s	remaining: 46.1s
442:	learn: 5.7518108	total: 36.6s	remaining: 46s
443:	learn: 5.7495561	total: 36.6s	remaining: 45.9s
444:	learn: 5.7465993	total: 36.7s	remaining: 45.8s
445:	learn: 5.7441230	total: 36.8s	remaining: 45.7s
446:	learn: 5.7411757	total: 36.9s	remaining: 45.6s
447:	learn: 5.7384810	total: 36.9s	remaining: 45.5s
448:	learn: 5.7366966	total: 37s	remaining: 45.4s
449:	learn: 5.7342830	

589:	learn: 5.4221676	total: 48.5s	remaining: 33.7s
590:	learn: 5.4208872	total: 48.6s	remaining: 33.6s
591:	learn: 5.4190504	total: 48.7s	remaining: 33.6s
592:	learn: 5.4162395	total: 48.9s	remaining: 33.5s
593:	learn: 5.4135076	total: 49s	remaining: 33.5s
594:	learn: 5.4121670	total: 49.2s	remaining: 33.5s
595:	learn: 5.4100898	total: 49.4s	remaining: 33.5s
596:	learn: 5.4077341	total: 49.4s	remaining: 33.4s
597:	learn: 5.4051338	total: 49.5s	remaining: 33.3s
598:	learn: 5.4025251	total: 49.6s	remaining: 33.2s
599:	learn: 5.4008766	total: 49.7s	remaining: 33.1s
600:	learn: 5.3986990	total: 49.8s	remaining: 33s
601:	learn: 5.3975062	total: 49.8s	remaining: 32.9s
602:	learn: 5.3955100	total: 49.9s	remaining: 32.9s
603:	learn: 5.3937545	total: 50s	remaining: 32.8s
604:	learn: 5.3919151	total: 50s	remaining: 32.7s
605:	learn: 5.3899300	total: 50.1s	remaining: 32.6s
606:	learn: 5.3878219	total: 50.2s	remaining: 32.5s
607:	learn: 5.3857798	total: 50.3s	remaining: 32.4s
608:	learn: 5.383070

750:	learn: 5.1273297	total: 1m 1s	remaining: 20.3s
751:	learn: 5.1256289	total: 1m 1s	remaining: 20.2s
752:	learn: 5.1238000	total: 1m 1s	remaining: 20.1s
753:	learn: 5.1226083	total: 1m 1s	remaining: 20s
754:	learn: 5.1211111	total: 1m 1s	remaining: 20s
755:	learn: 5.1202190	total: 1m 1s	remaining: 19.9s
756:	learn: 5.1189593	total: 1m 1s	remaining: 19.8s
757:	learn: 5.1169263	total: 1m 1s	remaining: 19.7s
758:	learn: 5.1151406	total: 1m 1s	remaining: 19.6s
759:	learn: 5.1129059	total: 1m 1s	remaining: 19.5s
760:	learn: 5.1117493	total: 1m 1s	remaining: 19.4s
761:	learn: 5.1092105	total: 1m 1s	remaining: 19.4s
762:	learn: 5.1074362	total: 1m 2s	remaining: 19.3s
763:	learn: 5.1060622	total: 1m 2s	remaining: 19.2s
764:	learn: 5.1042607	total: 1m 2s	remaining: 19.1s
765:	learn: 5.1026688	total: 1m 2s	remaining: 19s
766:	learn: 5.1011692	total: 1m 2s	remaining: 18.9s
767:	learn: 5.0993322	total: 1m 2s	remaining: 18.8s
768:	learn: 5.0977371	total: 1m 2s	remaining: 18.8s
769:	learn: 5.0959

908:	learn: 4.8813131	total: 1m 12s	remaining: 7.24s
909:	learn: 4.8795729	total: 1m 12s	remaining: 7.16s
910:	learn: 4.8780764	total: 1m 12s	remaining: 7.08s
911:	learn: 4.8773301	total: 1m 12s	remaining: 7s
912:	learn: 4.8757819	total: 1m 12s	remaining: 6.92s
913:	learn: 4.8745835	total: 1m 12s	remaining: 6.83s
914:	learn: 4.8733388	total: 1m 12s	remaining: 6.75s
915:	learn: 4.8717936	total: 1m 12s	remaining: 6.67s
916:	learn: 4.8702428	total: 1m 12s	remaining: 6.59s
917:	learn: 4.8688026	total: 1m 12s	remaining: 6.51s
918:	learn: 4.8674754	total: 1m 12s	remaining: 6.43s
919:	learn: 4.8664499	total: 1m 13s	remaining: 6.35s
920:	learn: 4.8640661	total: 1m 13s	remaining: 6.27s
921:	learn: 4.8624049	total: 1m 13s	remaining: 6.19s
922:	learn: 4.8608746	total: 1m 13s	remaining: 6.11s
923:	learn: 4.8598355	total: 1m 13s	remaining: 6.03s
924:	learn: 4.8581886	total: 1m 13s	remaining: 5.95s
925:	learn: 4.8567333	total: 1m 13s	remaining: 5.87s
926:	learn: 4.8556347	total: 1m 13s	remaining: 5.

In [155]:
y_pred1[y_pred1 <0] = 2 # 2 is better by checking local score on mse
y_pred2[y_pred2 <0] = 2
y_pred[y_pred <0] = 2


In [157]:
final_results = (y_pred2+y_pred1+y_pred)/3

# Saving results to the file

In [158]:
# Save the results to kaggle
df_test['hindex'] = pd.Series(np.round_(final_results, decimals=3))
df_test.loc[:,["author","hindex"]].to_csv('submission_second.csv', index=False)

In [ ]:
#What we tried, but it didn't work

import time
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(max_features = 200)
X_train_tfidf = tfidf.fit_transform(df_test_data['article'].to_numpy())
tdifd_arr = X_train_tfidf.toarray()

X_train_tfidf2 = tfidf.fit_transform(df_train_main['article_2'].to_numpy())
X_train_tfidf3 = tfidf.fit_transform(df_train_main['article_3'].to_numpy())
X_train_tfidf4 = tfidf.fit_transform(df_train_main['article_4'].to_numpy())
X_train_tfidf5 = tfidf.fit_transform(df_train_main['article_5'].to_numpy())

tdifd_arr = X_train_tfidf.toarray()
tdifd_arr2 = X_train_tfidf2.toarray()
tdifd_arr3 = X_train_tfidf3.toarray()
tdifd_arr4 = X_train_tfidf4.toarray()
tdifd_arr5 = X_train_tfidf5.toarray()

final_article1 = np.hstack((df_train_main[['degree', 'core']].to_numpy(), tdifd_arr,tdifd_arr2, tdifd_arr3, tdifd_arr4, tdifd_arr5 ))

